In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from utils import get_tx_from_nb, plot_horizontal_barchart
import plotly.express as px
import joblib 

In [2]:
path_data = Path('../data/')
path_df_stagiaire = path_data / 'df_stagiaire.pkl'
path_df_intervenant = path_data / "df_intervenant.pkl"
path_labels = path_data / "labels.pkl"
path_outputs = path_data / "outputs"
df = pd.read_pickle(path_df_stagiaire)
df_int = pd.read_pickle(path_df_intervenant)
with open(path_labels, "rb") as f: labels = joblib.load(f)

## contexte

In [3]:
nb_fi = len(df)
print(f"nombre d'étudiants en formation initiale: {nb_fi}")

nombre d'étudiants en formation initiale: 86


## analyse - satisfaction globale

In [4]:
cnt_sondage = dict()
cnt_sondage["Satisfaction globale"] = sum([df[f"Satisfaction globale.EV{i}"].value_counts() for i in range(1, 6)])
cnt_sondage["Organisation"] = sum([df[f"Organisation.EV{i}"].value_counts() for i in range(1, 6)])
cnt_sondage["Pédagogie"] = sum([df[f"Pédagogie.EV{i}"].value_counts() for i in range(1, 7)])
cnt_sondage["motivation"] = sum([df[f"motivation.EV{i}"].value_counts() for i in range(1, 7)])

for k in cnt_sondage:
    cnt_sondage[k] = list(cnt_sondage[k].replace(np.nan, 0).values)

res_sondage_nb = {"": cnt_sondage}
res_sondage_tx = get_tx_from_nb(res_sondage_nb)

In [5]:
cats = list(res_sondage_tx[""].keys())

i = 2
ic = True
iv = False
il = False
pct = True

top_labels = ['Très satisfait', 'Satisfait', 'Déçu', 'Très déçu']
print(cats)
title = f""
plot_horizontal_barchart(cats, res_sondage_tx[""], top_labels, inv_cats=ic, inv_values=iv, inv_labels=il, pct=pct, title=title)


['Satisfaction globale', 'Organisation', 'Pédagogie', 'motivation']


In [6]:
for c in ["MOYENNE_Satisfaction globale", "MOYENNE_Pédagogie", "MOYENNE_Organisation", "MOYENNE_motivation"]:
    print(f"{c}: {df[c].mean():.2f}")

MOYENNE_Satisfaction globale: 2.60
MOYENNE_Pédagogie: 2.46
MOYENNE_Organisation: 2.52
MOYENNE_motivation: 2.79


## analyse - par module

In [7]:
# Satisfaction globale, Motivation, Pédagogie, Organisation
modules = [f"module {i}" for i in range(1, 6)] + ["évaluation finale"]
mesures = ["Satisfaction globale", "motivation", "Pédagogie", "Organisation"]
mod_likert_nb = {mes:dict() for mes in mesures}

for mes in mesures:
    mod_likert_nb[mes] = dict()
    for i in range(1,6):
        mod = modules[i-1]
        col = f"{mes}.EV{i}"
        mod_likert_nb[mes][mod] = [(df[col] == n).sum() for n in range(1,5)]

    if mes in ["motivation", "Pédagogie"]:
        mod = modules[-1]
        col = f"{mes}.EV6"
        mod_likert_nb[mes][mod] = [(df[col] == n).sum() for n in range(1,5)]

In [8]:
cats = [f"module {i}" for i in range(1, 6)] + ["évaluation finale"]


include_EF = False
i = 2
ic = True
iv = False
il = False
pct = True

if not include_EF: cats = cats[:-1]

for mes in mesures:
    #mes = mesures[i]
    top_labels = ['Très satisfait', 'Satisfait', 'Déçu', 'Très déçu']
    mod_likert_tx = get_tx_from_nb(mod_likert_nb)
    print(mes)
    print(mod_likert_tx[mes])
    title = f"{mes} par module"
    plot_horizontal_barchart(cats, mod_likert_tx[mes], top_labels, inv_cats=ic, inv_values=iv, inv_labels=il, pct=pct, title=title)


Satisfaction globale
{'module 1': [0, 35, 53, 11], 'module 2': [3, 40, 48, 8], 'module 3': [3, 39, 50, 8], 'module 4': [5, 37, 52, 6], 'module 5': [10, 50, 32, 8]}


motivation
{'module 1': [0, 34, 37, 29], 'module 2': [15, 23, 35, 27], 'module 3': [16, 19, 32, 32], 'module 4': [15, 23, 31, 32], 'module 5': [18, 29, 27, 26], 'évaluation finale': [18, 37, 39, 6]}


Pédagogie
{'module 1': [27, 18, 23, 32], 'module 2': [27, 23, 23, 27], 'module 3': [23, 21, 40, 16], 'module 4': [29, 27, 21, 23], 'module 5': [29, 26, 26, 19], 'évaluation finale': [24, 24, 31, 21]}


Organisation
{'module 1': [13, 39, 19, 29], 'module 2': [24, 18, 42, 16], 'module 3': [23, 16, 42, 19], 'module 4': [21, 24, 39, 16], 'module 5': [24, 27, 37, 11]}


In [9]:
## tableau de performance par module
# taux de présence moyen
# nb stagiaires ayant assisté à l'ensemble des sessions présentielles du module
# nb d'abandons suite au module
# note moyenne (étudiants ayant participé au moins à 1 journée présentielle)
# (int) engagement des étudiants
# (int) satisfaction globale
# (int) satisfaction sur l'organisation
data_module = dict()
data_module["module"] = range(1, 6)

In [10]:
# taux de présence par module
data_module["tx_presence_module"] = [df[[f"PJ{i}" for i in range(1 + (imod-1)*3, 4 + (imod-1)*3)]].sum(axis=1).sum()/(3*len(df)) for imod in range(1, 6)]
data_module["tx_presence_module"] = [round(x*100)/100 for x in data_module["tx_presence_module"]]

In [11]:
# nombre de présents sur tout le module
#data_module["nb_present_tout_module"] = [df[f"present_tout_module_{i}"].sum() for i in range(1,6)]

In [12]:
## nombre d'abandons
data_module[f"nb_abandons_apres_module"] = [df[f"abandon_apres_module_{i}"].sum() for i in range(1, 6)]
print(f"% de la promotion abandonnant après la fin du premier module: {data_module['nb_abandons_apres_module'][0]/len(df):.0%}")

% de la promotion abandonnant après la fin du premier module: 13%


In [13]:
## note moyenne (étudiants ayant participé au moins à 1 journée présentielle)
note_moy = list()
for i in range(1, 6):
    b = ~df[f"absent_module_{i}"]
    moy = df[b][f"REV{i}"].mean()
    moy = round(moy*100)/100
    note_moy.append(moy)
data_module["note_moyenne"] = note_moy

In [14]:
## satisfaction des intervenants:
# satisfaction globale
# engagement des étudiants
# organisation du module
interv_engag = list()
interv_orga = list()
interv_satis = list()
for i in range(1, 6):
    interv_engag.append(df_int[f"Implication étudiants.EV{i}"].mean())
    interv_orga.append(df_int[f"Organisation.EV{i}"].mean())
    interv_satis.append(df_int[f"Satisfaction globale.EV{i}"].mean())
data_module["interv_engag"] = interv_engag
data_module["interv_orga"] = interv_orga
data_module["interv_satis"] = interv_satis

In [15]:
## ajout de la satisfaction des apprenants en moyenne sur chaque module
data_module["sta_motivation"] = [round(df[f"motivation.EV{i}"].mean()*100)/100 for i in range(1, 6)]
data_module["sta_peda"] = [round(df[f"Pédagogie.EV{i}"].mean()*100)/100 for i in range(1, 6)]
data_module["sta_orga"] = [round(df[f"Organisation.EV{i}"].mean()*100)/100 for i in range(1, 6)]
data_module["sta_global"] = [round(df[f"Satisfaction globale.EV{i}"].mean()*100)/100 for i in range(1, 6)]

In [16]:
df_data_module = pd.DataFrame(data_module)
df_data_module.to_excel(path_outputs / "df_module.xlsx")

In [17]:
df_data_module = pd.DataFrame(data_module)
df_data_module["tx_presence_module"] = df_data_module["tx_presence_module"] * 100
fig = px.line(data_frame=df_data_module, x="module", y="tx_presence_module", labels = {"tx_presence_module": "Taux de présence", "module": "Module"})
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
    ),
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 50,
        dtick = 10 
    ), 
    yaxis_ticksuffix = "%"
)
fig.update_xaxes(tickfont_size=20, title_font_size=30)
fig.update_yaxes(tickfont_size=20, title_font_size=30)
fig.show()

In [18]:
tx_pres_pef = df["PEF"].mean()
print(f"taux de présence à la journée d'évaluation finale: {tx_pres_pef:.0%}")

taux de présence à la journée d'évaluation finale: 84%


In [19]:
df_data_module.corr("spearman").style.applymap(lambda x: "color: red;")

,module,tx_presence_module,nb_abandons_apres_module,note_moyenne,interv_engag,interv_orga,interv_satis,sta_motivation,sta_peda,sta_orga,sta_global
module,1.000000,-0.700000,-0.461690,0.100000,-0.400000,-0.400000,-0.564288,-0.666886,-0.974679,-0.820783,-0.900000
tx_presence_module,-0.700000,1.000000,0.820783,0.100000,0.900000,0.900000,0.974679,0.820783,0.666886,0.666886,0.600000
nb_abandons_apres_module,-0.461690,0.820783,1.000000,-0.410391,0.718185,0.718185,0.789474,0.947368,0.552632,0.763158,0.615587
note_moyenne,0.100000,0.100000,-0.410391,1.000000,0.300000,0.300000,0.205196,-0.461690,-0.307794,-0.564288,-0.500000
interv_engag,-0.400000,0.900000,0.718185,0.300000,1.000000,1.000000,0.974679,0.666886,0.359092,0.410391,0.300000
interv_orga,-0.400000,0.900000,0.718185,0.300000,1.000000,1.000000,0.974679,0.666886,0.359092,0.410391,0.300000
interv_satis,-0.564288,0.974679,0.789474,0.205196,0.974679,0.974679,1.000000,0.763158,0.526316,0.552632,0.461690
sta_motivation,-0.666886,0.820783,0.947368,-0.461690,0.666886,0.666886,0.763158,1.000000,0.763158,0.921053,0.820783
sta_peda,-0.974679,0.666886,0.552632,-0.307794,0.359092,0.359092,0.526316,0.763158,1.000000,0.921053,0.974679
sta_orga,-0.820783,0.666886,0.763158,-0.564288,0.410391,0.410391,0.552632,0.921053,0.921053,1.000000,0.974679


## analyse - par absentéisme

In [23]:
# satisfaction globale, motivation, pédagogie, Organisation PAR absentéisme
mesures = ["Satisfaction globale", "motivation", "Pédagogie", "Organisation"]
pre_likert_nb = {mes:dict() for mes in mesures}

for mes in mesures:
    pre_likert_nb[mes] = dict()
    for pre in labels["presence"]:
        tmp = df.loc[df["pre_tx_lab"] == pre]
        pre_likert_nb[mes][pre] = [ sum([(tmp[f"{mes}.EV{i_mod}"] == i_lik).sum() for i_mod in range(1, 6)]) for i_lik in range(1, 5)]

In [24]:
## nombre de personnes par catégories
for pre in labels["presence"]:
    nb = (df["pre_tx_lab"] == pre).sum()
    print(f"Nombre de {pre}: {nb} ({nb/len(df):.0%})")

Nombre de absent(e): 17 (20%)
Nombre de intermittent(e): 22 (26%)
Nombre de assidu(e): 47 (55%)


In [26]:
## nombre de personnes par catégories
tx_rep_list = list()
for pre in labels["presence"]:
    b = df["pre_tx_lab"] == pre
    tx_reponse = (~df.loc[b]["motivation.EV1"].isna()).mean()
    tx_rep_list.append(tx_reponse)
    print(f"Taux de réponse aux questionnaire de satisfaction pour les {pre}: {nb} ({tx_reponse:.0%})")

df_tmp = pd.DataFrame({"profil_présence": labels["presence"], "taux_réponse_sondage": tx_rep_list})
fig = px.bar(df_tmp, x="taux_réponse_sondage", y="profil_présence", orientation='h')
fig.show()

Taux de réponse aux questionnaire de satisfaction pour les absent(e): 47 (76%)
Taux de réponse aux questionnaire de satisfaction pour les intermittent(e): 47 (59%)
Taux de réponse aux questionnaire de satisfaction pour les assidu(e): 47 (77%)


In [32]:
pre_likert_tx = get_tx_from_nb(pre_likert_nb)

cats = labels["presence"]
i = 3
ic = True
iv = False
il = False
pct = True

mes = mesures[i]
top_labels = labels["presence"]
print(mes)
print(pre_likert_tx[mes])
title = f"{mes} par taux de présence"
plot_horizontal_barchart(cats, pre_likert_tx[mes], top_labels, inv_cats=ic, inv_values=iv, inv_labels=il, pct=pct, title=title)


Organisation
{'absent(e)': [6, 20, 43, 31], 'intermittent(e)': [29, 26, 31, 14], 'assidu(e)': [23, 26, 35, 16]}


IndexError: list index out of range

In [33]:
## réussite académique par taux de présence
presence = labels["presence"]
pre_aca_nb = {"Réussite académique": dict()}
aca_labs = ["échec", "moyen", "bon", "excellent"]

for pre in presence:
    tmp = df.loc[df["pre_tx_lab"] == pre]
    pre_aca_nb["Réussite académique"][pre] = [ (tmp["academic_lab"] == aca).sum() for aca in aca_labs]
pre_aca_tx = get_tx_from_nb(pre_aca_nb)

In [34]:
## Score NPS par taux de présence
abs_nps_nb = {"Profil NPS": dict()}
for pre in labels["presence"]:
    b = (df["pre_tx_lab"] == pre)
    abs_nps_nb["Profil NPS"][pre] = [(df.loc[b]["NPS_LABEL"] == prf).sum() for prf in labels["NPS"]]
abs_nps_tx = get_tx_from_nb(abs_nps_nb)

In [35]:
cats = labels["presence"]
i = 3
ic = False
iv = True
il = True
pct = True

mes = "Profil NPS"
top_labels = labels["NPS"]
print(mes)
print(abs_nps_tx[mes])
title = f"{mes} par taux de présence"
plot_horizontal_barchart(cats, abs_nps_tx[mes], top_labels, inv_cats=ic, inv_values=iv, inv_labels=il, pct=pct, title=title)

Profil NPS
{'absent(e)': [69, 15, 15], 'intermittent(e)': [31, 31, 38], 'assidu(e)': [47, 36, 17]}


## score NPS

[Net promoter Score](https://fr.wikipedia.org/wiki/Net_Promoter_Score): estime la probabilité que les apprenants recommendent le cours à un ami ou collègue
- Les détracteurs (Score de 0 à 6)
- les passifs (NPS de 7 à 8)
- les promoteurs (NPS de 9 à 10)


In [36]:
print(f"Score NPS moyen: {df['NPS.EV6'].mean():.1f}")

Score NPS moyen: 6.4


In [39]:
profils_nps = dict()
nb_scores = df["NPS_LABEL"].isin(labels["NPS"]).sum()
for pr in labels["NPS"]:
    profils_nps[pr] = (df["NPS_LABEL"] == pr).sum()
    moy = profils_nps[pr]/nb_scores
    print(f"nombre de {pr}: {profils_nps[pr]} ({moy:.0%})")


nombre de detracteurs: 30 (48%)
nombre de passifs: 19 (31%)
nombre de promoteurs: 13 (21%)


## engagement sur plateforme

In [40]:
print(f"Taux d'engagement moyen sur la plateforme: {df['OC_engag_tx'].mean():.0%}")

Taux d'engagement moyen sur la plateforme: 56%


In [43]:
## lien entre engagement sur la plateforme et succès académique
data = {"engagement": labels["OC_engagement"]}
nb = [(df["OC_engag_label"] == e).sum() for e in labels["OC_engagement"]]
nb_pct = [f"{n} ({n/sum(nb):.0%})" for n in nb]
data["nombre"] = nb_pct

for aca in labels["perf_academique"]:
    b = (df["academic_lab"] == aca)
    data[aca] = [(df[b]["OC_engag_label"] == e).sum() for e in labels["OC_engagement"]]

df_eng = pd.DataFrame(data)
df_eng

,engagement,nombre,échec,moyen,bon,excellent
0,désengagé(e),26 (30%),22,3,0,1
1,moy. engagé(e),35 (41%),0,12,16,7
2,très engagé(e),25 (29%),0,0,9,16


In [44]:
pd.DataFrame({"engagement": df["OC_engag_tx"], "succès académique": df["MOYENNE_EV"]}).corr(method="spearman")

,engagement,succès académique
engagement,1.000000,0.890724
succès académique,0.890724,1.000000


Forte correlation entre l'**engagement** sur la plateforme et le **succès académique**
--> Utiliser la métrique d'engagement sur la plateforme de manière proactive durant la formation pour relancer les personnes qui en sont désengagée.

In [45]:
## lien entre fréquence de connexion et succès académique
data = {"fréquence de connection": labels["OC_frequence"]}
nb = [(df["OC_F_lab"] == e).sum() for e in labels["OC_frequence"]]
nb_pct = [f"{n} ({n/sum(nb):.0%})" for n in nb]
data["nombre"] = nb_pct

for aca in labels["perf_academique"]:
    b = (df["academic_lab"] == aca)
    data[aca] = [(df[b]["OC_F_lab"] == e).sum() for e in labels["OC_frequence"]]
    data[f"{aca}_pct"] = data[aca]/sum(data[aca])

df_eng = pd.DataFrame(data)
df_eng

,fréquence de connection,nombre,échec,échec_pct,moyen,moyen_pct,bon,bon_pct,excellent,excellent_pct
0,faible,45 (52%),22,1.0,1,0.066667,17,0.68,5,0.208333
1,moyenne,26 (30%),0,0.0,8,0.533333,8,0.32,10,0.416667
2,haute,15 (17%),0,0.0,6,0.400000,0,0.00,9,0.375000


In [46]:
pd.DataFrame({"fréquence de connection": df["OC_F"], "succès académique": df["MOYENNE_EV"]}).corr(method="spearman")

,fréquence de connection,succès académique
fréquence de connection,1.000000,0.441762
succès académique,0.441762,1.000000


Fréquence de connection est positivement corrélée avec le succès académique, mais moins que l'engagement sur la plateforme.
Hypothèse: une haute fréquence de connection peut refléter une **démarche active** de l'apprenant.
Cependant, les stratégies sont variées chez les excellents profils -> le plus important est de trouver sa propre manière d'apprendre

In [47]:
## lien entre engagement sur plateforme et présence
pd.DataFrame({"engagement": df["OC_engag_tx"], "présence": df["pre_tx"], "NPS": df["NPS.EV6"]}).corr(method="spearman")

,engagement,présence,NPS
engagement,1.000000,0.842971,0.085445
présence,0.842971,1.000000,0.083797
NPS,0.085445,0.083797,1.000000


## correlation des indicateurs

In [51]:
col_corr = [c for c in df.columns if c.startswith("MOYENNE_")]
col_corr += ["motivation.EV6", "Pédagogie.EV6","Insertionpro.EV6","NPS.EV6", "PEF"]
col_corr += ["pre_tx", "SUCCES", "OC_engag_tx"]
df_corr = df[col_corr].corr(method="spearman")
s = df_corr.style.applymap(highlight_corr)
s
df_corr.to_excel(path_outputs / "correlation.xlsx")

In [ ]:
## conclusions:
# meme échelle de lickert utilisée pour différentes métriques (si ce n'était pas le cas, correlation motivation, péda, et organisation, et satisfaction globale serait proche de 0)
# Likert corrèle negativement avec: MOyenne des évaluations, le score NPS, la présence à l'évaluation finale, le taux de présence, et l'engagement sur la plateforme
# Donc hypothèse: échelle décroissante. 1: Strongly agree, 2: agree, 3: disagree, 4: strongly disagree